In [128]:
import pandas as pd 
import numpy as np 
import os 
import requests
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [9]:
import requests
import pprint

teams = [
    
    "BOS",  # Boston Bruins
    "ANA",  # Anaheim Ducks
    "BUF",  # Buffalo Sabres
    "CAR",  # Carolina Hurricanes
    "CBJ",  # Columbus Blue Jackets
    "CGY",  # Calgary Flames
    "CHI",  # Chicago Blackhawks
    "COL",  # Colorado Avalanche
    "DAL",  # Dallas Stars
    "DET",  # Detroit Red Wings
                                     "EDM",  # Edmonton Oilers
    "FLA",  # Florida Panthers
    "LAK",  # Los Angeles Kings
    "MIN",  # Minnesota Wild
    "MTL",  # Montreal Canadiens
    "NJD",  # New Jersey Devils
    "NSH",  # Nashville Predators
    "NYI",  # New York Islanders
                                     "NYR",  # New York Rangers
    "OTT",  # Ottawa Senators
    "PHI",  # Philadelphia Flyers
    "PIT",  # Pittsburgh Penguins
    "SEA",  # Seattle Kraken
    "SJS",  # San Jose Sharks
    "STL",  # St. Louis Blues
    "TBL",  # Tampa Bay Lightning
    "TOR",  # Toronto Maple Leafs
    "UTA",  # Utah Hockey Club
                                    "VAN",  # Vancouver Canucks
    "VGK",  # Vegas Golden Knights
    "WPG",  # Winnipeg Jets
    "WSH",   # Washington Capitals
    "ARI",   # Arizona Coyotes
    "ATL",   # Atlanta Thrashers 
    "HFD",   # Hartford
    "PHX",   # Pheonix
    "QUE",   # Quebec
    "WIN",   # Winnepeg Jets - Old
     ]

seasons = [

    "20082009", "20092010",
    "20102011", "20112012", "20122013", "20132014", "20142015",
    "20152016", "20162017", "20172018", "20182019", "20192020",
    "20202021", "20212022", "20222023", "20232024", "20242025"
]

# Function to get player data from the NHL API
def get_player_data(team, season):
    url = f"https://api-web.nhle.com/v1/roster/{team}/{season}"
    response = requests.get(url)

    # Check if the response is valid JSON
    try:
        return response.json()
    except requests.JSONDecodeError:
        print(f"Error decoding JSON for team {team} in season {season}")
        return {}

# Initialize a dictionary to store player IDs
player_ids = {}

# Loop through each season in the seasons list
for season in seasons:
    player_ids[season] = {}

    # Loop through each team
    for team in teams:
        try:
            # Get the player data for the team and season
            player_data = get_player_data(team, season)
        
            # Initialize dictionaries for defencemen, forwards, and goalies
            player_ids[season][team] = {
                #'defencemen': [],
                #'forwards': [],
                'goalies': []
            }
        
            # # Extract player IDs from defencemen
            # if 'defensemen' in player_data:
            #     for player in player_data['defensemen']:
            #         player_id = player['id']
            #         player_ids[season][team]['defencemen'].append(player_id)
        
            # # Extract player IDs from forwards
            # if 'forwards' in player_data:
            #     for player in player_data['forwards']:
            #         player_id = player['id']
            #         player_ids[season][team]['forwards'].append(player_id)
        
            # Extract player IDs from goalies
            if 'goalies' in player_data:
                for player in player_data['goalies']:
                    player_id = player['id']
                    player_ids[season][team]['goalies'].append(player_id)
    
        except KeyError:
            print(f"Team {team} not found for season {season}. Skipping...")
            continue

#Print the player IDs
pprint.pprint(player_ids)

Error decoding JSON for team SEA in season 20082009
Error decoding JSON for team UTA in season 20082009
Error decoding JSON for team VGK in season 20082009
Error decoding JSON for team WPG in season 20082009
Error decoding JSON for team ARI in season 20082009
Error decoding JSON for team HFD in season 20082009
Error decoding JSON for team QUE in season 20082009
Error decoding JSON for team WIN in season 20082009
Error decoding JSON for team SEA in season 20092010
Error decoding JSON for team UTA in season 20092010
Error decoding JSON for team VGK in season 20092010
Error decoding JSON for team WPG in season 20092010
Error decoding JSON for team ARI in season 20092010
Error decoding JSON for team HFD in season 20092010
Error decoding JSON for team QUE in season 20092010
Error decoding JSON for team WIN in season 20092010
Error decoding JSON for team SEA in season 20102011
Error decoding JSON for team UTA in season 20102011
Error decoding JSON for team VGK in season 20102011
Error decodi

In [10]:
# Function to get game logs for a specific season
def get_game_log(player_id, season):
    url = f"https://api-web.nhle.com/v1/player/{player_id}/game-log/{season}/2"
    response = requests.get(url)
    return response.json()

# Check if the response is valid JSON
    try:
        return response.json()
    except requests.JSONDecodeError:
        print(f"Error decoding JSON for player {player_id} in season {season}")
        return {}

In [18]:
import pandas as pd

# Function to aggregate game logs for a specific player_id
def aggregate_game_logs(game_logs):
    aggregated_data = {}
    
    for game in game_logs:
        for key, value in game.items():
            if key not in aggregated_data:
                aggregated_data[key] = value
            else:
                # If the key already exists, aggregate the data (e.g., sum the values)
                if isinstance(value, (int, float)):
                    aggregated_data[key] += value
                elif isinstance(value, list):
                    aggregated_data[key].extend(value)
                # Handle other data types as needed
    
    return aggregated_data



# Initialize a dictionary to store DataFrames for each player_id
player_dfs = {}

# Loop through the selected player_ids
for season, teams in player_ids.items():
    for team, players in teams.items():
        for player_type, player_ids in players.items():
            for player_id in player_ids:
                game_log = get_game_log(player_id, season)
                
                if game_log and 'gameLog' in game_log:
                    game_log_data = game_log['gameLog']

                    seasons_list = []
                    
                    for stat in game_log_data:
                        stats_info = {   

                            'gameId': stat.get('gameId'), 
                            'homeRoadFlag': stat.get('homeRoadFlag'), 
                            'gameDate': stat.get('gameDate'), 
                            
                        
                        }
                        seasons_list.append(stats_info)
                    
                    # Create df from the flattened list of data
                    player_df = pd.DataFrame(seasons_list)

                    #  # Set gameDate as the index
                    # player_df.set_index('gameDate', inplace=True)

                    # Add the DataFrame to the player_dfs dictionary
                    player_dfs[player_id] = player_df

  
                   # Save the DataFrame to a CSV file
                    csv_filename = f'player_{player_id}.csv'
                    player_df.to_csv(csv_filename)
                    print(f"DataFrame for player_id {player_id} saved to {csv_filename}")
                else:
                    print(f"No valid data found for player_id {player_id} in season {season}")




DataFrame for player_id 8458988 saved to player_8458988.csv
DataFrame for player_id 8471695 saved to player_8471695.csv
DataFrame for player_id 8460703 saved to player_8460703.csv
DataFrame for player_id 8462044 saved to player_8462044.csv
DataFrame for player_id 8473972 saved to player_8473972.csv
DataFrame for player_id 8459579 saved to player_8459579.csv
DataFrame for player_id 8468011 saved to player_8468011.csv
DataFrame for player_id 8468550 saved to player_8468550.csv
DataFrame for player_id 8468038 saved to player_8468038.csv
DataFrame for player_id 8470320 saved to player_8470320.csv
DataFrame for player_id 8470584 saved to player_8470584.csv
DataFrame for player_id 8471305 saved to player_8471305.csv
DataFrame for player_id 8469461 saved to player_8469461.csv
DataFrame for player_id 8473461 saved to player_8473461.csv
DataFrame for player_id 8470090 saved to player_8470090.csv
DataFrame for player_id 8462147 saved to player_8462147.csv
DataFrame for player_id 8470147 saved to

In [19]:
# Function to load all CSV files into a dictionary of DataFrames
def load_all_csv_files(directory):
    csv_files = glob.glob(os.path.join(directory, '/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/nhlapigoalies/player_*.csv'))
    dataframes = {}
    
    for csv_file in csv_files:
        player_id = os.path.basename(csv_file).split('_')[1].split('.')[0]
        df = pd.read_csv(csv_file)
        dataframes[player_id] = df
    
    return dataframes

# Specify the directory where the CSV files are saved
directory = '.'  # Current directory

# Load all CSV files into a dictionary of DataFrames
player_dfs = load_all_csv_files(directory)

In [20]:
# Function to create a new DataFrame grouped by homeRoadFlag
def create_grouped_df(df):
    grouped_df = df.groupby('homeRoadFlag').agg({
        
        'gameId': 'first',
        'gameDate': 'first'
        
    }).rename(columns={'gameDate': 'gameCount'})
    # Sum the grouped statistics to create a single row DataFrame
    single_row_df = grouped_df.sum().to_frame().T
    
    return single_row_df

In [21]:
# Loop through each DataFrame and create a grouped DataFrame
grouped_dfs = {}
for player_id, df in player_dfs.items():
    try:
        # Check if 'gameDate' column has values
        if df['gameDate'].empty:
            print(f"No gameDate values found for player_id {player_id}. Skipping...")
            continue

        # Check if 'homeRoadFlag' column exists
        if 'homeRoadFlag' in df.columns:
            grouped_df = create_grouped_df(df)
            grouped_dfs[player_id] = grouped_df
        else:
            print(f"'homeRoadFlag' column not found for player_id {player_id}. Skipping...")
    except Exception as e:
        print(f"An error occurred for player_id {player_id}: {e}")

An error occurred for player_id 8484268: 'gameDate'
An error occurred for player_id 8483710: 'gameDate'
An error occurred for player_id 8481529: 'gameDate'
An error occurred for player_id 8482193: 'gameDate'
An error occurred for player_id 8480022: 'gameDate'
An error occurred for player_id 8482484: 'gameDate'
An error occurred for player_id 8483548: 'gameDate'
An error occurred for player_id 8482465: 'gameDate'
An error occurred for player_id 8480885: 'gameDate'
An error occurred for player_id 8480819: 'gameDate'
An error occurred for player_id 8481520: 'gameDate'


In [31]:
# Concatenate all DataFrames into a master DataFrame
master_goalie_df = pd.concat(grouped_dfs.values(), keys=grouped_dfs.keys(), names=['player_id', 'index'])

# Reset the index to have 'player_id' as a column
master_goalie_df.reset_index(level='index', drop=True, inplace=True)


In [48]:
master_goalie_df = master_goalie_df.drop(columns=['gameCount', 'gameId'])
master_goalie_df.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/master_copies/goalies.csv')

KeyError: "['gameCount', 'gameId'] not found in axis"

In [ ]:
goalies_all = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/goalies/goalie_quant_all.csv')
goalies_all.columns.to_list()

In [154]:
goalies_all = goalies_all.drop(columns=[
'GAA', 'SV%', 'SV',
 'Rk',
 'Born',
 'G',
 'A',
 'P',
 'PIM',
 'Unnamed: 17',
 'Unnamed: 18',
 'Unnamed: 19',
 'Unnamed: 20',
 'Unnamed: 21',
 'Unnamed: 22',
 'Unnamed: 23',
 'Unnamed: 24',
 'Unnamed: 25',
 'Unnamed: 26',
 'Unnamed: 27',
 'Unnamed: 28',
 'Unnamed: 29',
 'Unnamed: 30',
 'Unnamed: 31',
 'Unnamed: 32',
 'Unnamed: 33',
 'Unnamed: 34',
 'Unnamed: 35',
 'Unnamed: 36',
 'Unnamed: 37',
 'Unnamed: 38',
 'Unnamed: 39',
 'Unnamed: 40',
 'Unnamed: 41',
 'Unnamed: 42',
 'Unnamed: 43',
 'Unnamed: 44',
 'Unnamed: 45',
 'Unnamed: 46',
 'Unnamed: 47',
 'Unnamed: 48',
 'Unnamed: 49',
 'Unnamed: 50',
 'Unnamed: 51',
 'Unnamed: 52',
 'Unnamed: 53',
 'Unnamed: 54',
 'Unnamed: 55',
 'Unnamed: 56',
 'Unnamed: 57',
 'Unnamed: 58',
 'Unnamed: 59',
 'Unnamed: 60',
 'Unnamed: 61',
 'Unnamed: 62',
 'Unnamed: 63',
 'Unnamed: 64',
 'Unnamed: 65',
 'Unnamed: 66',
 'Unnamed: 67',
 'Unnamed: 68',
 'Unnamed: 69',
 'Unnamed: 70',
 'Unnamed: 71',
 'Unnamed: 72',
 'Unnamed: 73',
 'Unnamed: 74',
 'Unnamed: 75',
 'Unnamed: 76',
 'Unnamed: 77',
 'Unnamed: 78',
 'Unnamed: 79',
 'Unnamed: 80',
 'Unnamed: 81',
 'Unnamed: 82',
 'Unnamed: 83',
 'Unnamed: 84',
 'Unnamed: 85',
 'Unnamed: 86',
 'Unnamed: 87',
 'Unnamed: 88',
 'Unnamed: 89',
 'Unnamed: 90',
 'Unnamed: 91',
 'Unnamed: 92',
 'Unnamed: 93',
 'Unnamed: 94',
 'Unnamed: 95',
 'Unnamed: 96',
 'Unnamed: 97',
 'Unnamed: 98',
 'Unnamed: 99',
 'Unnamed: 100',
 'Unnamed: 101'])

In [ ]:
len(goalies_all)

In [ ]:
# Rename Name col
goalies_all = goalies_all.rename(columns={'Name': 'name'})

# Split the name_x column into first_name and last_name
def split_name(name):
    parts = name.split()
    if len(parts) > 1:
        return parts[-1], ' '.join(parts[:-1])
    else:
        return parts[0], ''

goalies_all[['last_name', 'first_name']] = goalies_all['name'].apply(lambda x: pd.Series(split_name(x)))

# Combine last_name and first_name into a single column with the format "Last, First"
goalies_all['name'] = goalies_all['last_name'] + ', ' + goalies_all['first_name']

# Drop the temporary first_name and last_name columns
goalies_all = goalies_all.drop(columns=['first_name', 'last_name'])

# Display the first few rows of the DataFrame to verify the change
goalies_all

In [ ]:
goalies_mp = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/goalies/goalies.csv')
goalies_mp.columns.to_list()

In [ ]:
# Split the name_x column into first_name and last_name
def split_name(name):
    parts = name.split()
    if len(parts) > 1:
        return parts[-1], ' '.join(parts[:-1])
    else:
        return parts[0], ''

goalies_mp[['last_name', 'first_name']] = goalies_mp['name'].apply(lambda x: pd.Series(split_name(x)))

# Combine last_name and first_name into a single column with the format "Last, First"
goalies_mp['name'] = goalies_mp['last_name'] + ', ' + goalies_mp['first_name']

# Drop the temporary first_name and last_name columns
goalies_mp = goalies_mp.drop(columns=['first_name', 'last_name'])

# Display the first few rows of the DataFrame to verify the change
goalies_mp

In [ ]:
goalies_mp = goalies_mp.drop(columns=['Unnamed: 0', 'icetime',
 'x_goals',
 'goals',
 'unblocked_shot_attempts',
 'x_rebounds',
 'rebounds',
 'x_freeze',
 'freeze',
 'x_on_goal',
 'on_goal',
 'x_play_stopped',
 'play_stopped',
 'x_play_continued_in_zone',
 'play_continued_in_zone',
 'x_play_continued_outside_zone',
 'play_continued_outside_zone',
 'flurry_adjusted_x_goals',
 'low_danger_shots',
 'medium_danger_shots',
 'high_danger_shots',
 'low_danger_x_goals',
 'medium_danger_x_goals',
 'high_danger_x_goals',
 'low_danger_goals',
 'medium_danger_goals',
 'high_danger_goals',
 'blocked_shot_attempts',
 'penalty_minutes',
 'penalties'])
goalies_mp

In [161]:
# Merge df
goalies_pred = pd.merge(goalies_mp, goalies_all, on='name', how='inner')


In [162]:
goalies_pred.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/linear_reg_csv_files/goalies_pred.csv')